# PETs/TETs – Hyperledger Aries – Manufacturer 2 (Holder) 💼

In [1]:
%%javascript
document.title ='💼 Manufacturer2 Agent'

<IPython.core.display.Javascript object>

## PART 2: Issue a VC to the Manufacturer Agents

**What:** Receive and store verifiable credential (VC) from the authority agent (i.e., the issuing authority)

**Why:** To prove that the agent is a manufacturer to relying parties (e.g., the city agent)

**How:** <br>
1. [Initiate Manufacturer's AgentCommunicationManager (ACM)](#1)
2. [Connect with the Authority agent](#2)
3. [Request VC](#3)


**Accompanying Agents and Notebooks:** 
* Authority 🏛️: `02_issue_VC_manufacturers.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import libs.helpers as helpers
from libs.agent_connection_manager import CredentialHolder
import os
from termcolor import colored

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

--- 

<a id=1></a>

### 1 – Initiate Manufacturer2 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer2-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer2_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC 
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer2_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': []}

---

<a id=2></a>

### 2 – Establish a connection with the Authority agent
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the Manufacturer2 requests a connection with the Authority to be certified as an official city agency. Thus, the Manufacturer2 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Create invitation to Authority agent


Several state changes of the connection between the Manufacturer agent, the inviter (A), and the authority agent, the invitee (B),  are required before successfully establishing a connection: 

| Step | State | Agent | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | invitation-sent | A | A sent an invitation to B | `create_connection_invitation()`
| 2 | invitation-received | B | B receives the invitation of A | Prompt: Paste invitation from A |
| 3 | request-sent | B | B sends B connection request | Prompt: Accept invitation OR `auto_accept=True` |
| 4 | request-received | A | A receives the connection request from B | Prompt: Accept invitation request response OR `auto_accept=True` |
| 5 | response-sent | A | A sends B response to B | - |
| 6 | response-received | B | B receives the response from A | - |
| 7 | active (completed) | A | B pings A to finalize connection | Prompt: Trust ping OR `auto_ping=True` |

In [8]:
# Setup for connection with Authority agent
alias = None
auto_accept = True # Accept response of Authority agent right away
auto_ping = True
public = False # Do not use public DID
multi_use = False # Invitation is only for one invitee

connection_id = manufacturer2_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, auto_ping=auto_ping)


Copy & paste invitation and share with external agent(s):
{
    '@id': '8fb0da73-38f3-4c8f-8aab-564a63f22695',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'AnonymousAgent2',
    'recipientKeys': ['42Yiy3JGeQ3uaMPwp6DCqmj8ytiTCAqUZcJxWqoeSx1h'],
    'serviceEndpoint': 'https://d2c37f56dbc8.ngrok.io',
}

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  ebda06c9-954f-44d1-b742-be15908c6c68
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  ebda06c9-954f-44d1-b742-be15908c6c68
State :  request (request-received)
Routing State : none
Connection with :  Governance-Authority
Their Role :  invitee
------------------

<div style="font-size: 25px"><center><b>Break Point 5</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 🏛</center></div><br>
<center>Please switch to the Authority agent 🏛: Please paste the invitation in Step 4.1 when prompted. If auto_accept is set to False, enter "yes" when prompted before proceeding.</center>

---

Proceed after Break Point 6
<a id=3></a>
### 3 – Request VC from `Authority` agent
#### 3.1 – Message `Authority` with request

In [10]:
# Get schema_id of manufacturer_schema to find the correct credential_exchange_id
identifiers = helpers.get_identifiers()
schema_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]

# Variables
auto_request = True # Authomatically requests VC when offer is found
auto_store = True # Automatically stores VC if issued by Authority

# Request VC
manufacturer2_agent.request_vc(connection_id, schema_id, auto_request, auto_store)

Get stored identifiers dictionary to access schema information 💾

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : ebda06c9-954f-44d1-b742-be15908c6c68
Credential exchange ID : 7507e537-7caf-43ce-8270-4b67cccf02b8
Agent Protocol Role :  holder
Protocol State :  request_sent
---------------------------------------------------------------------

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : ebda06c9-954f-44d1-b742-be15908c6c68
Credential exchange ID : 7507e537-7caf-43ce-8270-4b67cccf02b8
Agent Protocol Role :  holder
Protocol State :  credential_received
---------------------------------------------------------------------

Credential 7507e537-7caf-43ce-8270-4b67cccf02b8 is not stored in wallet.

Please provide a referent (Credential ID) for VC
(The referent acts as the identifier for retrieving th

Referent:  M2-isManufacturer-VC



Successfully stored credential (Credential ID: M2-isManufacturer-VC)

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : ebda06c9-954f-44d1-b742-be15908c6c68
Credential exchange ID : 7507e537-7caf-43ce-8270-4b67cccf02b8
Agent Protocol Role :  holder
Protocol State :  credential_acked
---------------------------------------------------------------------

Received Credential :
{
    'attrs': {
        'isManufacturer': 'TRUE',
        'manufacturerCity': 'Braunschweig',
        'manufacturerCountry': 'Germany',
        'manufacturerName': 'undisclosedManufacturer2',
    },
    'cred_def_id': 'XvWddwCGryjykggMo3vHrv:3:CL:251651:default',
    'referent': 'M2-isManufacturer-VC',
    'schema_id': 'XvWddwCGryjykggMo3vHrv:2:certify-manufacturer:0.0.1',
}


#### 3.3 – Verify that VC was stored in wallet

In [10]:
manufacturer2_agent.get_credentials()

{'results': [{'referent': 'M2-isManufacturer',
   'attrs': {'manufacturerCountry': 'Germany',
    'manufacturerCity': 'Braunschweig',
    'isManufacturer': 'TRUE',
    'manufacturerName': 'undisclosedManufacturer2'},
   'schema_id': 'au8Y1rjsZyz9jbpr3Lk54:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'au8Y1rjsZyz9jbpr3Lk54:3:CL:250893:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

--- 

### 4 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [11]:
await agent_controller.terminate()

<div style="font-size: 25px"><center><b>Break Point 7</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼</center></div><br>
<center><b>Please switch to the Manufacturer3 agent: Run the 02_get_manufacturer3_VC.ipynb notebook until Break Point 8</b></center>

---

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥